<a href="https://colab.research.google.com/github/SeohyunLee0321/DLstudy/blob/main/lab_11_6_PackedSequence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
from torch.nn.utils.rnn import pad_sequence, pack_sequence, pack_padded_sequence, pad_packed_sequence

In [ ]:
# Random word from random word generator
data = ['hello world',
        'midnight',
        'calculation',
        'path',
        'short circuit']

# Make dictionary
char_set = ['<pad>'] + list(set(char for seq in data for char in seq)) # Get all characters and include pad token
char2idx = {char: idx for idx, char in enumerate(char_set)} # Constuct character to index dictionary
print('char_set:', char_set)
print('char_set length:', len(char_set))

char_set: ['<pad>', 'i', 'g', 'h', 'o', 'p', ' ', 'e', 'r', 'd', 'm', 't', 'c', 'a', 'u', 's', 'n', 'w', 'l']
char_set length: 19


In [ ]:
# Convert character to index and make list of tensors
X = [torch.LongTensor([char2idx[char] for char in seq]) for seq in data]

# Check converted result
for sequence in X:
    print(sequence)

tensor([ 3,  7, 18, 18,  4,  6, 17,  4,  8, 18,  9])
tensor([10,  1,  9, 16,  1,  2,  3, 11])
tensor([12, 13, 18, 12, 14, 18, 13, 11,  1,  4, 16])
tensor([ 5, 13, 11,  3])
tensor([15,  3,  4,  8, 11,  6, 12,  1,  8, 12, 14,  1, 11])


In [ ]:
# Make length tensor (will be used later in 'pack_padded_sequence' function)
lengths = [len(seq) for seq in X]
print('lengths:', lengths)

lengths: [11, 8, 11, 4, 13]


In [ ]:
# Make a Tensor of shape (Batch x Maximum_Sequence_Length)
padded_sequence = pad_sequence(X, batch_first=True) # X is now padded sequence
print(padded_sequence)
print(padded_sequence.shape)

tensor([[ 3,  7, 18, 18,  4,  6, 17,  4,  8, 18,  9,  0,  0],
        [10,  1,  9, 16,  1,  2,  3, 11,  0,  0,  0,  0,  0],
        [12, 13, 18, 12, 14, 18, 13, 11,  1,  4, 16,  0,  0],
        [ 5, 13, 11,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [15,  3,  4,  8, 11,  6, 12,  1,  8, 12, 14,  1, 11]])
torch.Size([5, 13])


# pack_sequence 함수를 이용하여 PackedSequence 만들기

input (list of Tensor)는 길이에 따른 내림차순으로 정렬이 되어있어야

In [ ]:
# Sort by descending lengths
sorted_idx = sorted(range(len(lengths)), key=lengths.__getitem__, reverse=True)
sorted_X = [X[idx] for idx in sorted_idx]

# Check converted result
for sequence in sorted_X:
    print(sequence)

tensor([15,  3,  4,  8, 11,  6, 12,  1,  8, 12, 14,  1, 11])
tensor([ 3,  7, 18, 18,  4,  6, 17,  4,  8, 18,  9])
tensor([12, 13, 18, 12, 14, 18, 13, 11,  1,  4, 16])
tensor([10,  1,  9, 16,  1,  2,  3, 11])
tensor([ 5, 13, 11,  3])


pack_sequence를 이용하여 PackedSequence를 만들

In [ ]:
packed_sequence = pack_sequence(sorted_X)
print(packed_sequence)

PackedSequence(data=tensor([15,  3, 12, 10,  5,  3,  7, 13,  1, 13,  4, 18, 18,  9, 11,  8, 18, 12,
        16,  3, 11,  4, 14,  1,  6,  6, 18,  2, 12, 17, 13,  3,  1,  4, 11, 11,
         8,  8,  1, 12, 18,  4, 14,  9, 16,  1, 11]), batch_sizes=tensor([5, 5, 5, 5, 4, 4, 4, 4, 3, 3, 3, 1, 1]), sorted_indices=None, unsorted_indices=None)


# Embedding 적용해보기

one-hot character embedding

In [ ]:
# one-hot embedding using PaddedSequence
eye = torch.eye(len(char_set)) # Identity matrix of shape (len(char_set), len(char_set))
embedded_tensor = eye[padded_sequence]
print(embedded_tensor.shape) # shape: (Batch_size, max_sequence_length, number_of_input_tokens)

torch.Size([5, 13, 19])


In [ ]:
# one-hot embedding using PackedSequence
embedded_packed_seq = pack_sequence([eye[X[idx]] for idx in sorted_idx])
print(embedded_packed_seq.data.shape)

torch.Size([47, 19])


# RNN 모델 만들기

PaddedSequence를 이용하여 RNN에 넣기

In [ ]:
# declare RNN
rnn = torch.nn.RNN(input_size=len(char_set), hidden_size=30, batch_first=True)

rnn_output, hidden = rnn(embedded_tensor)
print(rnn_output.shape) # shape: (batch_size, max_seq_length, hidden_size)
print(hidden.shape)     # shape: (num_layers * num_directions, batch_size, hidden_size)

torch.Size([5, 13, 30])
torch.Size([1, 5, 30])


PackedSequence를 이용하여 RNN에 넣기

In [ ]:
rnn_output, hidden = rnn(embedded_packed_seq)
print(rnn_output.data.shape)
print(hidden.data.shape)

torch.Size([47, 30])
torch.Size([1, 5, 30])


In [ ]:
unpacked_sequence, seq_lengths = pad_packed_sequence(embedded_packed_seq, batch_first=True)
print(unpacked_sequence.shape)
print(seq_lengths)

torch.Size([5, 13, 19])
tensor([13, 11, 11,  8,  4])


PaddedSequence가 아까 길이에 따른 내림차순으로 정렬되어야 하기 때문에 재정렬

In [ ]:
embedded_padded_sequence = eye[pad_sequence(sorted_X, batch_first=True)]
print(embedded_padded_sequence.shape)

torch.Size([5, 13, 19])


padding이 된 Tensor를 PackedSequence로 변환

In [ ]:
sorted_lengths = sorted(lengths, reverse=True)
new_packed_sequence = pack_padded_sequence(embedded_padded_sequence, sorted_lengths, batch_first=True)
print(new_packed_sequence.data.shape)
print(new_packed_sequence.batch_sizes)

torch.Size([47, 19])
tensor([5, 5, 5, 5, 4, 4, 4, 4, 3, 3, 3, 1, 1])
